In [1]:
#라이브러리 호출
import pandas as pd
import numpy as np
import time
import random

In [2]:
#외부 파일 호출
df = pd.read_csv('2023공모전데이터.csv')
df = df.sort_values(by='Unnamed: 0')
df = df.drop(['Unnamed: 0'], axis=1)

In [3]:
#필요한 데이터 전처리
#컬럼 리스트
column_list = df.columns

#학습 날짜 리스트
date_list = list(set(df['Date']))

#발전소 리스트
all_power_code_list = list(set(df['Power_Code']))

#중형 발전소 리스트
all_middle_power_code_list = list(set(df[df['Power_Volume(kw)']>1000]['Power_Code']))

#소형 발전소 리스트
all_small_power_code_list = list(set(df[df['Power_Volume(kw)']<=1000]['Power_Code']))

In [4]:
#필요한 함수 정의
#리스트 중에 중형 발전소를 찾아서 발전량 리스트로 반환
def fun_find_middle_volumn(list) :
    middle_volumn_list = [] 
    for power_code in list : 
        for middle_power_code in all_middle_power_code_list : 
            if power_code == middle_power_code : 
                middle_volumn_list.append(df[df['Power_Code']==power_code]['Power_Volume(kw)'].iloc[0])
                break
    return middle_volumn_list

#리스트 중에 소형 발전소를 찾아서 발전량 리스트로 반환
def fun_find_small_volumn(list) :
    small_volumn_list = [] 
    for power_code in list : 
        for small_power_code in all_small_power_code_list : 
            if power_code == small_power_code : 
                small_volumn_list.append(df[df['Power_Code']==power_code]['Power_Volume(kw)'].iloc[0])
                break
    return small_volumn_list

#정산금 측정을 위해 소형 발전량, 중형 발전량을 찾아주는 함수
def fun_small_middle_volumn(list) : 
    middle_volumn_list = fun_find_middle_volumn(list)
    small_volumn_list = fun_find_small_volumn(list)
    middle_volumn = sum(middle_volumn_list)
    small_volumn = sum(small_volumn_list)
    return small_volumn, middle_volumn

#정산금 측정을 위해 시간별 합산 실측 발전량과, 합산 예측 발전량을 구하는 함수
def fun_time_real_pred_power(list, date, time):
    power_real = 0
    power_pred = 0
    for power_code in list : 
        selected = df[df['Power_Code']== power_code]
        power_real += selected[selected['Date'] == date][column_list[time+3]].iloc[0]
        power_pred += selected[selected['Date'] == date][column_list[time+28]].iloc[0]
    return power_real, power_pred

#해당 시간의 이용률과 svf를 구하는 함수
def svf_fun(power_real, total_volume) : 
    cfs = (power_real / total_volume) * 100
    if cfs >= 10 : 
        svf = 1
    else : svf = 0
    return svf

#해당 시간의 정산금을 구하는 함수
def price_time_fun(power_real, power_pred, svf, total_volumn):
    error_time = abs(power_real*svf - power_pred*svf)*100/total_volumn #해당 시간의 오차율
    if error_time <= 6:
        price_rate = 4
    elif error_time <= 8 : 
        price_rate = 3
    else : price_rate = 0
    price_time = power_real*svf*price_rate
    return price_time

#발전소 리스트를 입력하면 최종적으로 정산금을 출력하는 함수
def fun_price(list) : 
    price = 0
    small_volumn, middle_volumn = fun_small_middle_volumn(list)
    total_volumn = small_volumn + middle_volumn

    # #1) 소형 발전량 > 중형 발전량 검사
    # if small_volumn > middle_volumn : 

    #2) 총 발전량 20초과 검사    
    if total_volumn > 20000 :         
        #날짜 별 계산
        for date in date_list : 
            price_day = 0
            #시간 별 계산
            for time in range(0,24,1) : 
                #리스트 시간 별 합산 실측 발전량, 합산 예측 발전량
                power_real, power_pred = fun_time_real_pred_power(list, date, time)                  
                svf = svf_fun(power_real, total_volumn) #해당 시간의 svf        
                if svf > 0: 
                    price_time = int(price_time_fun(power_real, power_pred, svf, total_volumn) )
                    price_day += price_time
            price += price_day
    return price         

#발전소 리스트를 입력하면 최대 정산금을 출력하는 함수
def fun_best_price(list) : 
    price = 0
    small_volumn, middle_volumn = fun_small_middle_volumn(list)
    total_volumn = small_volumn + middle_volumn
      
    #날짜 별 계산
    for date in date_list : 
        price_day = 0
        #시간 별 계산
        for time in range(0,24,1) : 
            #리스트 시간 별 합산 실측 발전량, 합산 예측 발전량
            power_real, power_pred = fun_time_real_pred_power(list, date, time)                  
            price_time = power_real*6
            price_day += price_time
        price += price_day
    return price         

def fun_random_group_list(all_power_code_list,group) : 
    power_code_list_random = all_power_code_list
    random.shuffle(power_code_list_random)
    
    group_index_list = []
    for i in range(0,group,1) : 
        random_index = random.randint(1,len(all_power_code_list)-1)
        group_index_list.append(random_index)
    group_index_list.sort()

    #랜덤한 크기만큼 발전소 코드 인덱스를 분할
    result_list = []
    group_index_first = 0
    if group > 0 :
        for i in range(0,len(group_index_list)+1,1) : 
            if i < len(group_index_list) : 
                group_index = group_index_list[i]
                result_list.append(power_code_list_random[group_index_first:group_index])
                group_index_first = group_index
            else : 
                result_list.append(power_code_list_random[group_index_list[i-1]:])
    else:
        result_list = [power_code_list_random]   
    return result_list

In [7]:
#1차 출력
start = time.time()
#출력용 데이터프레임 생성
data_dict = {'Result_Number':[], 'Total_Money':[], 'Total_Best_Money':[], 'Money_percent':[], 'Number_k':[],
'Power_Code_list':[], 'Total_Volumn_list':[], 'Middle_Volumn_list':[], 'Small_Volumn_list':[], 'Price_list':[]}
df_result = pd.DataFrame(data_dict)


#파라미터 설정
train_num = 1 #학습 횟수
group_num = 4 #그룹 수
num = 0
#리스트 분할을 위한 알고리즘
for group in range(1, group_num,1):
    for train in range(0,train_num,1) : 
        num+=1
        maximum_train_num = train_num*group_num-train_num
        print(num,'/',maximum_train_num)
        
        #발전소 코드 리스트를 무작위로 셔플 후 그룹 수 만큼 분할
        result_list = fun_random_group_list(all_power_code_list, group)
        
        #출력용 데이터 프레임에 값 넣기
        middle_volumn_list = []
        small_volumn_list = []
        total_volumn_list = []
        price_list = []
        power_code_list = []
        
        for j in range(0,len(result_list),1):
            list = result_list[j]

            price = fun_price(list)
            small_volumn, middle_volumn = fun_small_middle_volumn(list)
            total_volumn = small_volumn + middle_volumn

            power_code_list.append(list)
            middle_volumn_list.append(round(middle_volumn,2))
            small_volumn_list.append(round(small_volumn,2))
            total_volumn_list.append(round(total_volumn,2))
            price_list.append(int(price))
        best_price = 106115586


        newdata = {'Result_Number':num, 'Total_Money':sum(price_list), 'Total_Best_Money':best_price, 'Money_percent':sum(price_list)/best_price, 'Number_k':group+1,
                    'Power_Code_list':power_code_list, 'Total_Volumn_list':total_volumn_list, 
                    'Middle_Volumn_list':middle_volumn_list, 'Small_Volumn_list':small_volumn_list, 'Price_list':price_list}
        df_result=df_result.append(newdata, ignore_index=True)

df_result = df_result.sort_values('Total_Money', ascending=False)
# df_result = df_result.drop(['Unnamed: 0'], axis=1)

end = time.time()
print(f"{end - start:.5f} sec")
df_result_filename = '사후분석-'+str(group_num)+'그룹_'+str(maximum_train_num)+'횟수'+'_'+str(round((end-start)/60,0))+'분.csv'
df_result.to_csv(df_result_filename)

1 / 3
2 / 3
3 / 3
305.34351 sec
